In [15]:
# Run this again after editing submodules so Colab uses the updated versions
from citylearn import  CityLearn
from pathlib import Path
from agent import Agent
import numpy as np                                                                                                                                                                                      
import torch

In [11]:
# Load environment
climate_zone = 5
params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
        'building_attributes':'building_attributes.json', 
        'weather_file':'weather_data.csv', 
        'solar_profile':'solar_generation_1kW.csv', 
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
        'buildings_states_actions':'buildings_state_action_space.json', 
        'simulation_period': (0, 8760*4-1), 
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
        'central_agent': False,
        'save_memory': False }

# Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
env = CityLearn(**params)
observations_spaces, actions_spaces = env.get_state_action_spaces()

# Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
building_info = env.get_building_information()

In [13]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
                 'buildings_states_actions':'buildings_state_action_space.json', 
                 'building_info':building_info,
                 'observation_spaces':observations_spaces, 
                 'action_spaces':actions_spaces}

# Instantiating the control agent(s)
agents = Agent(**params_agent)

state = env.reset()
done = False

action, coordination_vars = agents.select_action(state)    
while not done:
    next_state, reward, done, _ = env.step(action)
    action_next, coordination_vars_next = agents.select_action(next_state)
    agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)
    coordination_vars = coordination_vars_next
    state = next_state
    action = action_next

env.cost()

Device:cuda


({'ramping': 1.6146312,
  '1-load_factor': 1.1158112003254197,
  'average_daily_peak': 1.1334853,
  'peak_demand': 1.1599934,
  'net_electricity_consumption': 1.0157715,
  'carbon_emissions': 1.0229925,
  'total': 1.177114183933326,
  'coordination_score': 1.2559802759152476},
 {'ramping_last_yr': 1.1639097,
  '1-load_factor_last_yr': 1.0558413421324109,
  'average_daily_peak_last_yr': 1.0668069,
  'peak_demand_last_yr': 1.1332848,
  'net_electricity_consumption_last_yr': 1.0071987,
  'carbon_emissions_last_yr': 1.0151038,
  'coordination_score_last_yr': 1.1049606838626467,
  'total_last_yr': 1.0781579896667743})